In [6]:
from itertools import combinations
from binance.helpers import round_step_size
from binance.client import Client
from binance.enums import *
from time import strftime
from datetime import datetime, timedelta, date, time
import itertools
import pandas as pd
import numpy as np
import warnings
import os.path
import time
import os
import ta

warnings.filterwarnings("ignore")


########### KEYS ###########
MASTER_API_KEY='PFL78Ps4WnSGGhZRcDQ3AFkzBPko9jA2B3d7lasLgDBI8VREV0QdHPYWaVuSzint'
MASTER_API_SEC='wO9zNzEK7LmgJ8bfcnkh60miN785lWaA4S34XydNMEzq18e90d5fTK92vbdIRYg2'


In [7]:

class GET_BALANCE():
    def __init__(self, client):
        self.client             = client
        self.BALANCE            = self.GET_BALANCE()

    def GET_BALANCE(self):
        acc_balance = self.client.futures_account_balance()
        for check_balance in acc_balance:
            if check_balance["asset"] == "USDT":
                BALANCE = round(float(check_balance["balance"]),2)

        return BALANCE




class OPEN_POSITIONS():

    def __init__(self, client, COIN):
        self.client                                                     = client
        self.COIN                                                       = COIN
        self.POS_DF, self.ENTRY_PRICE, self.DIRECTION_EXIST, self.QTY,  = self.OPEN_POSITIONS()


    def OPEN_POSITIONS(self):

        QTY, DIRECTION_EXIST, ENTRY_PRICE     = 0, 'NA', 0

        try:

            POS_DF                                = pd.DataFrame(self.client.futures_account()['positions'])
            POS_DF                                = POS_DF[POS_DF['symbol'] == self.COIN]
            POS_DF['entryPrice']                  = POS_DF['entryPrice'].astype(float)
            POS_DF                                = POS_DF[POS_DF['entryPrice'] > 0]

            if len(POS_DF) > 0:
                POS_DF               = POS_DF[POS_DF['symbol'] == self.COIN].reset_index()
                ENTRY_PRICE          = POS_DF.at[0, 'entryPrice']
                DIRECTION_EXIST      = float(POS_DF.at[0, 'positionAmt'])
                QTY                  = abs(float(POS_DF.at[0, 'positionAmt']))

                if DIRECTION_EXIST < 0:
                    DIRECTION_EXIST = 'SELL'
                else:
                    DIRECTION_EXIST = 'BUY'

        except:
            print('UNABLE -- Unable to get POS_DF - Assuming no positions open')
            POS_DF = []

        return POS_DF, ENTRY_PRICE, DIRECTION_EXIST, QTY



class SIZE_2():
    def __init__(self, client, COIN):
        self.client                    = client
        self.COIN                      = COIN
        self.STEP_SIZE, self.TICK_SIZE = self.SIZE_2()

    def SIZE_2(self):

        info = self.client.futures_exchange_info()
        info = info['symbols']
        for x in range(len(info)):
            if info[x]['symbol'] == self.COIN:

                return info[x]['quantityPrecision'], info[x]['pricePrecision']
        return None




class CHECK_IF_POS_OPEN():

    def __init__(self, ROOT):
        self.ROOT              = ROOT
        self.POS_OPEN           = self.CHECK_IF_POS_OPEN()

    def CHECK_IF_POS_OPEN(self):

        POS_OPEN = False

        with open((self.ROOT) + "/CHECK_POS.txt", "r") as f:
            contents = f.readlines()
            POS_OPEN = contents

        return POS_OPEN



In [8]:


class CLOSE_POSITION():
    def __init__(self, CLIENT, TICKER, QTY, DIRECTION_EXIST):
        self.CLIENT             = CLIENT
        self.TICKER             = TICKER
        self.QTY                = QTY
        self.DIRECTION_EXIST    = DIRECTION_EXIST
        self.CLOSE_POSITION()

    def CLOSE_POSITION(self):
        if self.DIRECTION_EXIST == 'SELL':      DIRECTION = 'BUY'
        else:                                   DIRECTION = 'SELL'
        self.CLIENT.futures_create_order(symbol=self.TICKER, side=DIRECTION, type='MARKET', quantity=self.QTY)

        return



class CREATE_ORDER():
    def __init__(self, CLIENT, TICKER, QTY, DIRECTION, LEVERAGE):
        self.CLIENT             = CLIENT
        self.TICKER             = TICKER
        self.QTY                = QTY
        self.DIRECTION          = DIRECTION
        self.LEVERAGE           = LEVERAGE
        self.CREATE_ORDER()

    def CREATE_ORDER(self):
        if self.DIRECTION == 'SHORT':       SIDE      = 'SELL'
        else:                               SIDE      = 'BUY'
        
        self.CLIENT.futures_change_leverage(symbol=self.TICKER, leverage=self.LEVERAGE)
        self.CLIENT.futures_create_order(symbol=self.TICKER, side=SIDE, type='MARKET', quantity=self.QTY)

        return


class CREATE_TP_AND_SL():
    def __init__(self, CLIENT, TICKER, TP, SL, TICK_SIZE):
        self.CLIENT                 = CLIENT
        self.TICKER                 = TICKER
        self.TP                     = TP
        self.SL                     = SL
        self.TICK_SIZE              = TICK_SIZE
        self.CREATE_TP_AND_SL()


    def CREATE_TP_AND_SL(self):
        time.sleep(3)

        OPEN_POS                                    = OPEN_POSITIONS(self.CLIENT, self.TICKER)
        if OPEN_POS.DIRECTION_EXIST == 'BUY':       SIDE = 'SELL'
        else:                                       SIDE = 'BUY'
        
        TAKE_PROFIT_PRICE                           = "{:0.0{}f}".format(self.TP, self.TICK_SIZE) 
        STOP_LIMIT_PRICE                            = "{:0.0{}f}".format(self.SL, self.TICK_SIZE)

        if  TAKE_PROFIT_PRICE != 0:
            try:                                    self.CLIENT.futures_create_order(symbol=self.TICKER, side=SIDE, type='STOP_MARKET', timeInForce= 'GTE_GTC', stopPrice=STOP_LIMIT_PRICE, closePosition='true')
            except:                                 pass
        if  STOP_LIMIT_PRICE != 0:
            try:                                    self.CLIENT.futures_create_order(symbol=self.TICKER, side=SIDE, type='TAKE_PROFIT_MARKET', timeInForce= 'GTE_GTC', stopPrice=TAKE_PROFIT_PRICE, closePosition='true')
            except:                                 pass





In [9]:


class HISTORICAL_DATA():
    def __init__(self, client, TICKER, TIMEFRAME):
        self.client         = client
        self.TICKER         = TICKER
        self.TIMEFRAME      = TIMEFRAME
        self.DF             = self.get_data()

    def get_data(self):

        INTERVALS   =  {'1M'    : {'PERIODS': 1,        'FUNCTION': self.client.KLINE_INTERVAL_1MINUTE},
                        '5M'    : {'PERIODS': 2,        'FUNCTION': self.client.KLINE_INTERVAL_5MINUTE},
                        '15M'   : {'PERIODS': 4,        'FUNCTION': self.client.KLINE_INTERVAL_15MINUTE},
                        '30M'   : {'PERIODS': 6,        'FUNCTION': self.client.KLINE_INTERVAL_30MINUTE},
                        '1H'    : {'PERIODS': 11,       'FUNCTION': self.client.KLINE_INTERVAL_1HOUR},
                        '2H'    : {'PERIODS': 21,       'FUNCTION': self.client.KLINE_INTERVAL_2HOUR},
                        '4H'    : {'PERIODS': 42,       'FUNCTION': self.client.KLINE_INTERVAL_4HOUR},
                        '6H'    : {'PERIODS': 63,       'FUNCTION': self.client.KLINE_INTERVAL_6HOUR},
                        '8H'    : {'PERIODS': 84,       'FUNCTION': self.client.KLINE_INTERVAL_8HOUR},
                        '12H'   : {'PERIODS': 125,      'FUNCTION': self.client.KLINE_INTERVAL_12HOUR},
                        '1D'    : {'PERIODS': 250,      'FUNCTION': self.client.KLINE_INTERVAL_1DAY},
                        '1W'    : {'PERIODS': 1750,     'FUNCTION': self.client.KLINE_INTERVAL_1WEEK},
                    }

        PERIODS                 = INTERVALS[self.TIMEFRAME]['PERIODS']
        FUNCTIONS               = INTERVALS[self.TIMEFRAME]['FUNCTION']
        END                     = (datetime.now() + timedelta(days = 1)).date()
        START                   = END - timedelta(days = PERIODS)

        candle                  = (np.asarray(self.client.get_historical_klines(self.TICKER, FUNCTIONS, str(START), str(END))))[:, :6]
        candle                  = pd.DataFrame(candle, columns=['datetime', 'open', 'high', 'low', 'close', 'volume']).astype(float).rename(columns={'datetime':'DATE', 'open':'OPEN', 'high':'HIGH', 'low':'LOW', 'close':'CLOSE', 'volume':'VOLUME'})
        candle.DATE             = pd.to_datetime(candle.DATE, unit='ms')
        return candle








class TECHNICAL_ANALYSIS:
    def __init__(self, DF):
        self.DF = DF
        self.DF, self.INDICATOR_OUTPUT = self.TECHNICAL_ANALYSIS()

    def TECHNICAL_ANALYSIS(self):
        TA_PARAMS_WIDER_SCOPE                       = { 'EMA_NBRS': [4, 8, 12, 18, 24, 36, 50, 100, 200],
                                                        'SMA_NBRS': [4, 8, 12, 18, 24, 36, 50, 100, 200],
                                                        'RSI': [14, 20, 30],
                                                        'ATR_PERIOD': [10],
                                                        'ATR_MULTIPLIER': [0.35]}

        allIndicators                               = { 'EMA'                   : [f'EMA_{x}' for x in TA_PARAMS_WIDER_SCOPE['EMA_NBRS']],
                                                        'SMA'                   : [f'SMA_{x}' for x in TA_PARAMS_WIDER_SCOPE['SMA_NBRS']],
                                                        'RSI'                   : [f'RSI_{x}' for x in TA_PARAMS_WIDER_SCOPE['RSI']],
                                                        'ADX'                   : ['ADX_NEG', 'ADX_POS'],  
                                                        'MACD'                  : ['MACD', 'MACD_SIGNAL', 'MACD_DIFF'],              
                                                        'STOCHASTIC_OSCILLATOR' : ['%K', '%D'],                                                    
                                                        'BOLLINGER_BANDS'       : ['BB_UPPER', 'BB_MIDDLE', 'BB_LOWER'],  
                                                        'PARABOLIC_SAR'         : ['PSAR'],
                                                        'ATR'                   : [f'ATR_{x[0]}_{x[1]}' for x in itertools.product(TA_PARAMS_WIDER_SCOPE['ATR_PERIOD'], TA_PARAMS_WIDER_SCOPE['ATR_MULTIPLIER'])],
                                                        'MISC_SEPERATE'         : ['CCI', 'WILLIAMS_%R', 'TSI', 'MFI', 'ROC'],
                                                        'MISC_OVERLAY'          : ['VWAP']}

        indicatorMap                                = { "OVERLAY"               : ["EMA", "SMA", "BOLLINGER_BANDS", 'MISC_OVERLAY'],
                                                        "SEPARATE"              : ["RSI", "STOCHASTIC_OSCILLATOR", "MACD", "ADX", "MISC_SEPERATE"],
                                                        "PRICE"                 : ["PARABOLIC_SAR", "ATR"],}

        INDICATOR_OUTPUT                            = {'allIndicators' : allIndicators, 'indicatorMap' : indicatorMap}
        for k in TA_PARAMS_WIDER_SCOPE['EMA_NBRS']: self.DF[f'EMA_{k}'] = self.DF['CLOSE'].ewm(span=k, adjust=False).mean()
        for k in TA_PARAMS_WIDER_SCOPE['SMA_NBRS']: self.DF[f'SMA_{k}'] = self.DF['CLOSE'].rolling(window=k).mean()
        for k in TA_PARAMS_WIDER_SCOPE['RSI']:      self.DF[f'RSI_{k}'] = ta.momentum.RSIIndicator(self.DF['CLOSE'], window=k).rsi()
        ATR_OPTIONS                                 = list(itertools.product(TA_PARAMS_WIDER_SCOPE['ATR_PERIOD'], TA_PARAMS_WIDER_SCOPE['ATR_MULTIPLIER']))
        for period, multiplier in ATR_OPTIONS:      self.DF[f'ATR_{period}_{multiplier}'] = (ta.volatility.AverageTrueRange(high=self.DF['HIGH'], low=self.DF['LOW'], close=self.DF['CLOSE'], window=period).average_true_range() * multiplier).fillna(0)

        macd                                        = ta.trend.MACD(self.DF['CLOSE'])
        self.DF['MACD']                             = macd.macd()
        self.DF['MACD_SIGNAL']                      = macd.macd_signal()
        self.DF['MACD_DIFF']                        = macd.macd_diff()

        bollinger                                   = ta.volatility.BollingerBands(self.DF['CLOSE'], window=20, window_dev=2)
        self.DF['BB_UPPER']                         = bollinger.bollinger_hband()
        self.DF['BB_MIDDLE']                        = bollinger.bollinger_mavg()
        self.DF['BB_LOWER']                         = bollinger.bollinger_lband()
        self.DF['BB_WIDTH']                         = self.DF['BB_UPPER'] - self.DF['BB_LOWER']

        self.DF['VWAP']                             = (self.DF['VOLUME'] * (self.DF['HIGH'] + self.DF['LOW'] + self.DF['CLOSE']) / 3).cumsum() / self.DF['VOLUME'].cumsum()
        self.DF['CCI']                              = ta.trend.CCIIndicator(high=self.DF['HIGH'], low=self.DF['LOW'], close=self.DF['CLOSE'], window=20).cci()
        self.DF['WILLIAMS_%R']                      = ta.momentum.WilliamsRIndicator(high=self.DF['HIGH'], low=self.DF['LOW'], close=self.DF['CLOSE'], lbp=14).williams_r()
        adx                                         = ta.trend.ADXIndicator(high=self.DF['HIGH'], low=self.DF['LOW'], close=self.DF['CLOSE'], window=14)
        self.DF['ADX']                              = adx.adx()
        self.DF['ADX_NEG']                          = adx.adx_neg()
        self.DF['ADX_POS']                          = adx.adx_pos()
        self.DF['PSAR']                             = ta.trend.PSARIndicator(high=self.DF['HIGH'], low=self.DF['LOW'], close=self.DF['CLOSE']).psar()
        self.DF['TSI']                              = ta.momentum.TSIIndicator(self.DF['CLOSE']).tsi()
        self.DF['MFI']                              = ta.volume.MFIIndicator(high=self.DF['HIGH'], low=self.DF['LOW'], close=self.DF['CLOSE'], volume=self.DF['VOLUME'], window=14).money_flow_index()
        self.DF['ROC']                              = ta.momentum.ROCIndicator(self.DF['CLOSE'], window=12).roc()
        STOCH                                       = ta.momentum.StochasticOscillator(self.DF['HIGH'], self.DF['LOW'], self.DF['CLOSE'])
        self.DF['%K']                               = (STOCH.stoch())
        self.DF['%D']                               = (STOCH.stoch_signal())


        self.DF['PREV_GAIN_UP']                     = ((self.DF['HIGH'] / self.DF['OPEN']) - 1).shift(1)
        self.DF['PREV_GAIN_DOWN']                   = ((self.DF['OPEN'] / self.DF['LOW']) - 1).shift(1)
        self.DF                                     = self.DF.iloc[200:, :].reset_index(drop=True)

        return self.DF, INDICATOR_OUTPUT













In [10]:
class STRATEGY():
    def __init__(self, DATA_DF, INPUT_PARAMETERS):
        self.DATA_DF            = DATA_DF
        self.INPUT_PARAMETERS   = INPUT_PARAMETERS
        self.DF                 = self.STRATEGY()

    def STRATEGY(self):
        TAKE_PROFIT = 0.02
        STOP_LOSS   = 0.04
        
        self.DATA_DF['SIGNAL']          = np.where((self.DATA_DF['EMA_18'].shift(1) <= self.DATA_DF['EMA_24'].shift(1)) & (self.DATA_DF['EMA_18'] > self.DATA_DF['EMA_24']), "LONG",  
                                          np.where((self.DATA_DF['EMA_18'].shift(1) >= self.DATA_DF['EMA_24'].shift(1)) & (self.DATA_DF['EMA_18'] < self.DATA_DF['EMA_24']), "SHORT", "STATIC"))
        self.DATA_DF['TP']              = np.where((self.DATA_DF['SIGNAL'] == 'LONG'), (self.DATA_DF['OPEN'] * (1 + TAKE_PROFIT)), np.where((self.DATA_DF['SIGNAL'] == 'SHORT'), self.DATA_DF['OPEN'] * (1 - TAKE_PROFIT), 0))
        self.DATA_DF['SL']              = np.where((self.DATA_DF['SIGNAL'] == 'LONG'), (self.DATA_DF['OPEN'] * (1 - STOP_LOSS)), np.where((self.DATA_DF['SIGNAL'] == 'SHORT'), self.DATA_DF['OPEN'] * (1 + STOP_LOSS), 0))

        return self.DATA_DF


In [11]:
class ACTION():
    def __init__(self, STRATEGY_LIST, CONDITIONS, UTILIZATION, LEVERAGE, CLIENT):
        self.STRATEGY_LIST          = STRATEGY_LIST
        self.CONDITIONS             = CONDITIONS
        self.UTILIZATION            = UTILIZATION
        self.LEVERAGE               = LEVERAGE
        self.CLIENT                 = CLIENT
        self.ACTION()

    def ACTION(self):

        for a in range(len(self.STRATEGY_LIST)):
            COIN                    = self.CONDITIONS['COIN'][a]
            ENTRY                   = self.STRATEGY_LIST[a].at[len(self.STRATEGY_LIST[a])-1, 'OPEN']
            ACTION                  = self.STRATEGY_LIST[a].at[len(self.STRATEGY_LIST[a])-1, 'SIGNAL']
            TP                      = self.STRATEGY_LIST[a].at[len(self.STRATEGY_LIST[a])-1, 'TP']
            SL                      = self.STRATEGY_LIST[a].at[len(self.STRATEGY_LIST[a])-1, 'SL']
            if ACTION != 'STATIC':  self.OPEN_NEW_POSITION(COIN, ENTRY, ACTION, TP, SL)

        for a in range(len(self.STRATEGY_LIST)):
            print('\nCOIN REVIEWED - ' + self.CONDITIONS['COIN'][a] + '    DECISION - ' + self.STRATEGY_LIST[a].at[len(self.STRATEGY_LIST[a])-1, 'SIGNAL'])
            print(self.STRATEGY_LIST[a].tail())


    def OPEN_NEW_POSITION(self, COIN, ENTRY, ACTION, TP, SL):
        STEP_INFO                   = SIZE_2(self.CLIENT, COIN)
        CURRENT                     = GET_BALANCE(self.CLIENT)
        BALANCE_TO_TRADE            = (CURRENT.BALANCE*self.UTILIZATION) * self.LEVERAGE
        QTY                         = "{:0.0{}f}".format((BALANCE_TO_TRADE/ENTRY), STEP_INFO.STEP_SIZE)

        CREATE_ORDER(self.CLIENT, COIN, QTY, ACTION, self.LEVERAGE)
        CREATE_TP_AND_SL(self.CLIENT, COIN, TP,  SL, STEP_INFO.TICK_SIZE)

        print('\nORDER OPENED - ' + COIN + ' - ' + ACTION + ' - Position opened at : ' + str(round(ENTRY, 6)) + "\n")







In [ ]:


class CONTROLLER():

    def __init__(self, CONDITIONS, TIMEFRAME, UTILIZATION, LEVERAGE, CLIENT):
        self.CONDITIONS                         = CONDITIONS
        self.TIMEFRAME                          = TIMEFRAME
        self.UTILIZATION                        = UTILIZATION
        self.LEVERAGE                           = LEVERAGE
        self.CLIENT                             = CLIENT
        self.ACTIONS, self.STRATEGY             = self.CONTROLLER()


    def CONTROLLER(self):

        STRATEGY_LIST = []

        for a in range(len(self.CONDITIONS['COIN'])):

            RAW                                 = HISTORICAL_DATA(self.CLIENT, self.CONDITIONS['COIN'][a], self.TIMEFRAME)
            TA_DF                               = TECHNICAL_ANALYSIS(RAW.DF)
            STRATEGY_DF                         = STRATEGY(TA_DF.DF, self.CONDITIONS['INPUT_PARAMS'][a])

            STRATEGY_LIST.append(STRATEGY_DF.DF)


        STATUS = ACTION(STRATEGY_LIST, self.CONDITIONS, self.UTILIZATION, self.LEVERAGE, self.CLIENT)


        return STRATEGY_LIST




In [ ]:




CONDITIONS                  = {'COIN'           :  {0      : 'XRPUSDT',
                                                    1      : 'DOGEUSDT',
                                                    2      : 'ADAUSDT'},

                               'INPUT_PARAMS'   :  {0      : [2, 3, 4],
                                                    1      : [1],
                                                    2      : [1, 3]}}


TIMEFRAME       = '2H'
UTILIZATION     = 0.95
LEVERAGE        = 5



In [ ]:

def FULL_RUN():

    #CLIENT                         = Client(os.getenv("SUB_API_KEY_2"), os.getenv("SUB_API_SEC_2"))

    MASTER_API_KEY                  = 'PFL78Ps4WnSGGhZRcDQ3AFkzBPko9jA2B3d7lasLgDBI8VREV0QdHPYWaVuSzint'
    MASTER_API_SEC                  = 'wO9zNzEK7LmgJ8bfcnkh60miN785lWaA4S34XydNMEzq18e90d5fTK92vbdIRYg2'
    CLIENT                          = Client(MASTER_API_KEY, MASTER_API_SEC)

    CHECK                           = CHECK_IF_POS_OPEN((r'/home/ENKIINVESTMENTS/CRYPTO_TRADER/ALGO_TRADING_2'))
    if CHECK.PROCEED[0] == 'True':  DATA = CONTROLLER(CONDITIONS, TIMEFRAME, UTILIZATION, LEVERAGE, CLIENT)
    else:                           print('POSITION CURRENTLY OPEN')

    return











if __name__ == '__main__':
    #project_folder = os.path.expanduser('~/CRYPTO_TRADER/ALGO_TRADING_2/')
    #load_dotenv(os.path.join(project_folder, '.env'))

    FULL_RUN(CONDITIONS, TIMEFRAME, UTILIZATION, LEVERAGE)


In [ ]:


def FULL_RUN():

    STRATEGY_1_CONDITIONS       = { 'COIN'            :  {0      : 'XRPUSDT',
                                                          1      : 'DOGEUSDT',
                                                          2      : 'ADAUSDT'},

                                    'INPUT_PARAMS'    :  {0      : [2, 3, 4],
                                                          1      : [1],
                                                          2      : [1, 3]},

                                    'TAKE_PROFIT'     :  {0      : 0.75,
                                                          1      : 0.75,
                                                          2      : 0.75},

                                    'STOP_LOSS'       :  {0      : 18,
                                                          1      : 18,
                                                          2      : 18}}






    client                      = Client(os.getenv("SUB_API_KEY_2"), os.getenv("SUB_API_SEC_2"))
    DONE                        = 'NO'


    while DONE == 'NO':
        while DONE == 'NO':
            if datetime.now() >= (datetime.now().replace(minute=58, second=0, microsecond=0)) and datetime.now() <= (datetime.now().replace(minute=59, second=4, microsecond=0)):
                CHECK                       = CHECK_IF_POS_OPEN((r'/home/ENKIINVESTMENTS/CRYPTO_TRADER/ALGO_TRADING_2'))
                break


        while DONE == 'NO':
            if datetime.now() >= (datetime.now().replace(minute=59, second=40, microsecond=0)) and datetime.now() <= (datetime.now().replace(minute=59, second=59, microsecond=0)):
                if CHECK.POS_OPEN[0] == 'False':

                        COIN                    = DATA_RUN(STRATEGY_1_CONDITIONS, client)
                        STATUS                  = ACTION_M1(STRATEGY_1_CONDITIONS, COIN.ACTIONS, COIN.STRATEGY, client)

                else:                       print('POSITION CURRENTLY OPEN')
                break

        break

    return











if __name__ == '__main__':
    project_folder = os.path.expanduser('~/CRYPTO_TRADER/ALGO_TRADING_2/')
    load_dotenv(os.path.join(project_folder, '.env'))
    FULL_RUN()


